In [1]:
pip install tensorflow numpy matplotlib opencv-python scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, Add, Multiply, Concatenate, UpSampling2D, Dense, GlobalAveragePooling2D, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split

# Custom data generator
class CustomDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, sar_files, optical_files, batch_size, img_size=(256, 256)):
        self.sar_files = sar_files
        self.optical_files = optical_files
        self.batch_size = batch_size
        self.img_size = img_size
        self.indices = np.arange(len(self.sar_files))

    def __len__(self):
        return int(np.floor(len(self.sar_files) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        sar_batch = [self.sar_files[i] for i in batch_indices]
        optical_batch = [self.optical_files[i] for i in batch_indices]
        return self.__data_generation(sar_batch, optical_batch)

    def __data_generation(self, sar_batch, optical_batch):
        sar_images = np.array([img_to_array(load_img(file, target_size=self.img_size)) / 255.0 for file in sar_batch])
        optical_images = np.array([img_to_array(load_img(file, target_size=self.img_size)) / 255.0 for file in optical_batch])
        return [tf.convert_to_tensor(sar_images), tf.convert_to_tensor(optical_images)], tf.convert_to_tensor(optical_images)

# Load dataset file paths
def load_dataset_paths(base_path):
    print(f"Executing load_dataset_paths for base_path: {base_path}")
    sar_files = []
    optical_files = []
    for category in ['agri', 'barrenland', 'grassland', 'urban']:
        category_path = os.path.join(base_path, category)
        sar_path = os.path.join(category_path, 's1')
        optical_path = os.path.join(category_path, 's2')
        
        sar_files.extend([os.path.join(sar_path, file) for file in os.listdir(sar_path)])
        optical_files.extend([os.path.join(optical_path, file) for file in os.listdir(optical_path)])
    
    return sar_files, optical_files

# Squeeze-and-Excitation (SE) Block
def se_block(input_tensor, reduction=16):
    print("Executing se_block")
    filters = input_tensor.shape[-1]
    se = GlobalAveragePooling2D()(input_tensor)
    se = Dense(filters // reduction, activation='relu')(se)
    se = Dense(filters, activation='sigmoid')(se)
    se = Reshape((1, 1, filters))(se)
    se = Multiply()([input_tensor, se])
    return se

# Attention Block
def attention_block(input_tensor, gating_tensor):
    print("Executing attention_block")
    filters = input_tensor.shape[-1]
    gating = Conv2D(filters, 1, activation='relu', padding='same')(gating_tensor)
    gating = Conv2D(filters, 1, activation='sigmoid', padding='same')(gating)
    attention = Multiply()([input_tensor, gating])
    return attention

# Define Advanced U-Net with Fusion and Attention
def advanced_unet_with_fusion(input_shape):
    print("Executing advanced_unet_with_fusion")
    inputs_sar = Input(shape=input_shape)
    inputs_optical = Input(shape=input_shape)  # Optical data
    
    # Fusion Layer
    fusion = Concatenate()([inputs_sar, inputs_optical])
    
    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(fusion)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    conv1 = se_block(conv1)
    pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    conv2 = se_block(conv2)
    pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # Bottleneck
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    conv3 = se_block(conv3)
    
    # Decoder with Attention
    up4 = UpSampling2D(size=(2, 2))(conv3)
    concat4 = Concatenate()([up4, conv2])
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(concat4)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(conv4)
    conv4 = se_block(conv4)
    
    up5 = UpSampling2D(size=(2, 2))(conv4)
    concat5 = Concatenate()([up5, conv1])
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(concat5)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(conv5)
    
    # Output: LAB color space (3 channels: L, A, B)
    outputs = Conv2D(3, 1, activation='tanh')(conv5)
    
    model = Model(inputs=[inputs_sar, inputs_optical], outputs=outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

# Load the data (SAR and optical)
print("Loading the data (SAR and optical)")
base_path = 'v_2'
sar_files, optical_files = load_dataset_paths(base_path)

# Train/test split
print("Splitting the data into train and validation sets")
X_train_sar, X_val_sar, X_train_optical, X_val_optical = train_test_split(
    sar_files, optical_files, test_size=0.2, random_state=42
)

# Define and train the model
print("Defining and training the model")
input_shape = (256, 256, 3)  # Assuming RGB images of size 256x256
model = advanced_unet_with_fusion(input_shape)

# Create data generators
batch_size = 16
train_generator = CustomDataGenerator(X_train_sar, X_train_optical, batch_size)
val_generator = CustomDataGenerator(X_val_sar, X_val_optical, batch_size)

# Train the model
print("Training the model")
model.fit(train_generator, validation_data=val_generator, epochs=20)

# Save the trained model
print("Saving the trained model")
model.save('advanced_sar_colorization_model.h5')

Loading the data (SAR and optical)
Executing load_dataset_paths for base_path: v_2
Splitting the data into train and validation sets
Defining and training the model
Executing advanced_unet_with_fusion
Executing se_block
Executing se_block
Executing se_block
Executing se_block
Training the model


TypeError: `output_signature` must contain objects that are subclass of `tf.TypeSpec` but found <class 'list'> which is not.

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, Add, Multiply, Concatenate, UpSampling2D, Dense, GlobalAveragePooling2D, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split

# Custom data generator
class CustomDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, sar_files, optical_files, batch_size, img_size=(256, 256)):
        self.sar_files = sar_files
        self.optical_files = optical_files
        self.batch_size = batch_size
        self.img_size = img_size
        self.indices = np.arange(len(self.sar_files))

    def __len__(self):
        return int(np.floor(len(self.sar_files) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        sar_batch = [self.sar_files[i] for i in batch_indices]
        optical_batch = [self.optical_files[i] for i in batch_indices]
        return self.__data_generation(sar_batch, optical_batch)

    def __data_generation(self, sar_batch, optical_batch):
        sar_images = np.array([img_to_array(load_img(file, target_size=self.img_size)) / 255.0 for file in sar_batch])
        optical_images = np.array([img_to_array(load_img(file, target_size=self.img_size)) / 255.0 for file in optical_batch])
        return [tf.convert_to_tensor(sar_images), tf.convert_to_tensor(optical_images)], tf.convert_to_tensor(optical_images)

# Load dataset file paths
def load_dataset_paths(base_path):
    print(f"Executing load_dataset_paths for base_path: {base_path}")
    sar_files = []
    optical_files = []
    for category in ['agri', 'barrenland', 'grassland', 'urban']:
        category_path = os.path.join(base_path, category)
        sar_path = os.path.join(category_path, 's1')
        optical_path = os.path.join(category_path, 's2')
        
        sar_files.extend([os.path.join(sar_path, file) for file in os.listdir(sar_path)])
        optical_files.extend([os.path.join(optical_path, file) for file in os.listdir(optical_path)])
    
    return sar_files, optical_files

# Squeeze-and-Excitation (SE) Block
def se_block(input_tensor, reduction=16):
    print("Executing se_block")
    filters = input_tensor.shape[-1]
    se = GlobalAveragePooling2D()(input_tensor)
    se = Dense(filters // reduction, activation='relu')(se)
    se = Dense(filters, activation='sigmoid')(se)
    se = Reshape((1, 1, filters))(se)
    se = Multiply()([input_tensor, se])
    return se

# Attention Block
def attention_block(input_tensor, gating_tensor):
    print("Executing attention_block")
    filters = input_tensor.shape[-1]
    gating = Conv2D(filters, 1, activation='relu', padding='same')(gating_tensor)
    gating = Conv2D(filters, 1, activation='sigmoid', padding='same')(gating)
    attention = Multiply()([input_tensor, gating])
    return attention

# Define Advanced U-Net with Fusion and Attention
def advanced_unet_with_fusion(input_shape):
    print("Executing advanced_unet_with_fusion")
    inputs_sar = Input(shape=input_shape)
    inputs_optical = Input(shape=input_shape)  # Optical data
    
    # Fusion Layer
    fusion = Concatenate()([inputs_sar, inputs_optical])
    
    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(fusion)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    conv1 = se_block(conv1)
    pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    conv2 = se_block(conv2)
    pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # Bottleneck
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    conv3 = se_block(conv3)
    
    # Decoder with Attention
    up4 = UpSampling2D(size=(2, 2))(conv3)
    concat4 = Concatenate()([up4, conv2])
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(concat4)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(conv4)
    conv4 = se_block(conv4)
    
    up5 = UpSampling2D(size=(2, 2))(conv4)
    concat5 = Concatenate()([up5, conv1])
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(concat5)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(conv5)
    
    # Output: LAB color space (3 channels: L, A, B)
    outputs = Conv2D(3, 1, activation='tanh')(conv5)
    
    model = Model(inputs=[inputs_sar, inputs_optical], outputs=outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

# Load the data (SAR and optical)
print("Loading the data (SAR and optical)")
base_path = 'v_2'
sar_files, optical_files = load_dataset_paths(base_path)

# Train/test split
print("Splitting the data into train and validation sets")
X_train_sar, X_val_sar, X_train_optical, X_val_optical = train_test_split(
    sar_files, optical_files, test_size=0.2, random_state=42
)

# Define and train the model
print("Defining and training the model")
input_shape = (256, 256, 3)  # Assuming RGB images of size 256x256
model = advanced_unet_with_fusion(input_shape)

# Create data generators
batch_size = 16
train_generator = CustomDataGenerator(X_train_sar, X_train_optical, batch_size)
val_generator = CustomDataGenerator(X_val_sar, X_val_optical, batch_size)

# Train the model
print("Training the model")
model.fit(train_generator, validation_data=val_generator, epochs=20)

# Save the trained model
print("Saving the trained model")
model.save('advanced_sar_colorization_model.h5')

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, Add, Multiply, Concatenate, UpSampling2D, Dense, GlobalAveragePooling2D, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split

# Custom data generator
class CustomDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, sar_files, optical_files, batch_size, img_size=(256, 256)):
        self.sar_files = sar_files
        self.optical_files = optical_files
        self.batch_size = batch_size
        self.img_size = img_size
        self.indices = np.arange(len(self.sar_files))

    def __len__(self):
        return int(np.floor(len(self.sar_files) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        sar_batch = [self.sar_files[i] for i in batch_indices]
        optical_batch = [self.optical_files[i] for i in batch_indices]
        return self.__data_generation(sar_batch, optical_batch)

    def __data_generation(self, sar_batch, optical_batch):
        sar_images = np.array([img_to_array(load_img(file, target_size=self.img_size)) / 255.0 for file in sar_batch])
        optical_images = np.array([img_to_array(load_img(file, target_size=self.img_size)) / 255.0 for file in optical_batch])
        return [tf.convert_to_tensor(sar_images), tf.convert_to_tensor(optical_images)], tf.convert_to_tensor(optical_images)

# Load dataset file paths
def load_dataset_paths(base_path):
    print(f"Executing load_dataset_paths for base_path: {base_path}")
    sar_files = []
    optical_files = []
    for category in ['agri', 'barrenland', 'grassland', 'urban']:
        category_path = os.path.join(base_path, category)
        sar_path = os.path.join(category_path, 's1')
        optical_path = os.path.join(category_path, 's2')
        
        sar_files.extend([os.path.join(sar_path, file) for file in os.listdir(sar_path)])
        optical_files.extend([os.path.join(optical_path, file) for file in os.listdir(optical_path)])
    
    return sar_files, optical_files

# Squeeze-and-Excitation (SE) Block
def se_block(input_tensor, reduction=16):
    print("Executing se_block")
    filters = input_tensor.shape[-1]
    se = GlobalAveragePooling2D()(input_tensor)
    se = Dense(filters // reduction, activation='relu')(se)
    se = Dense(filters, activation='sigmoid')(se)
    se = Reshape((1, 1, filters))(se)
    se = Multiply()([input_tensor, se])
    return se

# Attention Block
def attention_block(input_tensor, gating_tensor):
    print("Executing attention_block")
    filters = input_tensor.shape[-1]
    gating = Conv2D(filters, 1, activation='relu', padding='same')(gating_tensor)
    gating = Conv2D(filters, 1, activation='sigmoid', padding='same')(gating)
    attention = Multiply()([input_tensor, gating])
    return attention

# Define Advanced U-Net with Fusion and Attention
def advanced_unet_with_fusion(input_shape):
    print("Executing advanced_unet_with_fusion")
    inputs_sar = Input(shape=input_shape)
    inputs_optical = Input(shape=input_shape)  # Optical data
    
    # Fusion Layer
    fusion = Concatenate()([inputs_sar, inputs_optical])
    
    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(fusion)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    conv1 = se_block(conv1)
    pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    conv2 = se_block(conv2)
    pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # Bottleneck
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    conv3 = se_block(conv3)
    
    # Decoder with Attention
    up4 = UpSampling2D(size=(2, 2))(conv3)
    concat4 = Concatenate()([up4, conv2])
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(concat4)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(conv4)
    conv4 = se_block(conv4)
    
    up5 = UpSampling2D(size=(2, 2))(conv4)
    concat5 = Concatenate()([up5, conv1])
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(concat5)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(conv5)
    
    # Output: LAB color space (3 channels: L, A, B)
    outputs = Conv2D(3, 1, activation='tanh')(conv5)
    
    model = Model(inputs=[inputs_sar, inputs_optical], outputs=outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

# Load the data (SAR and optical)
print("Loading the data (SAR and optical)")
base_path = 'v_2'
sar_files, optical_files = load_dataset_paths(base_path)

# Train/test split
print("Splitting the data into train and validation sets")
X_train_sar, X_val_sar, X_train_optical, X_val_optical = train_test_split(
    sar_files, optical_files, test_size=0.2, random_state=42
)

# Define and train the model
print("Defining and training the model")
input_shape = (256, 256, 3)  # Assuming RGB images of size 256x256
model = advanced_unet_with_fusion(input_shape)

# Create data generators
batch_size = 16
train_generator = CustomDataGenerator(X_train_sar, X_train_optical, batch_size)
val_generator = CustomDataGenerator(X_val_sar, X_val_optical, batch_size)

# Train the model
print("Training the model")
model.fit(train_generator, validation_data=val_generator, epochs=20)

# Save the trained model
print("Saving the trained model")
model.save('advanced_sar_colorization_model.h5')

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, Add, Multiply, Concatenate, UpSampling2D, Dense, GlobalAveragePooling2D, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split

# Custom data generator
class CustomDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, sar_files, optical_files, batch_size, img_size=(256, 256)):
        self.sar_files = sar_files
        self.optical_files = optical_files
        self.batch_size = batch_size
        self.img_size = img_size
        self.indices = np.arange(len(self.sar_files))

    def __len__(self):
        return int(np.floor(len(self.sar_files) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        sar_batch = [self.sar_files[i] for i in batch_indices]
        optical_batch = [self.optical_files[i] for i in batch_indices]
        return self.__data_generation(sar_batch, optical_batch)

    def __data_generation(self, sar_batch, optical_batch):
        sar_images = np.array([img_to_array(load_img(file, target_size=self.img_size)) / 255.0 for file in sar_batch])
        optical_images = np.array([img_to_array(load_img(file, target_size=self.img_size)) / 255.0 for file in optical_batch])
        return [tf.convert_to_tensor(sar_images), tf.convert_to_tensor(optical_images)], tf.convert_to_tensor(optical_images)

# Load dataset file paths
def load_dataset_paths(base_path):
    print(f"Executing load_dataset_paths for base_path: {base_path}")
    sar_files = []
    optical_files = []
    for category in ['agri', 'barrenland', 'grassland', 'urban']:
        category_path = os.path.join(base_path, category)
        sar_path = os.path.join(category_path, 's1')
        optical_path = os.path.join(category_path, 's2')
        
        sar_files.extend([os.path.join(sar_path, file) for file in os.listdir(sar_path)])
        optical_files.extend([os.path.join(optical_path, file) for file in os.listdir(optical_path)])
    
    return sar_files, optical_files

# Squeeze-and-Excitation (SE) Block
def se_block(input_tensor, reduction=16):
    print("Executing se_block")
    filters = input_tensor.shape[-1]
    se = GlobalAveragePooling2D()(input_tensor)
    se = Dense(filters // reduction, activation='relu')(se)
    se = Dense(filters, activation='sigmoid')(se)
    se = Reshape((1, 1, filters))(se)
    se = Multiply()([input_tensor, se])
    return se

# Attention Block
def attention_block(input_tensor, gating_tensor):
    print("Executing attention_block")
    filters = input_tensor.shape[-1]
    gating = Conv2D(filters, 1, activation='relu', padding='same')(gating_tensor)
    gating = Conv2D(filters, 1, activation='sigmoid', padding='same')(gating)
    attention = Multiply()([input_tensor, gating])
    return attention

# Define Advanced U-Net with Fusion and Attention
def advanced_unet_with_fusion(input_shape):
    print("Executing advanced_unet_with_fusion")
    inputs_sar = Input(shape=input_shape)
    inputs_optical = Input(shape=input_shape)  # Optical data
    
    # Fusion Layer
    fusion = Concatenate()([inputs_sar, inputs_optical])
    
    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(fusion)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    conv1 = se_block(conv1)
    pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    conv2 = se_block(conv2)
    pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # Bottleneck
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    conv3 = se_block(conv3)
    
    # Decoder with Attention
    up4 = UpSampling2D(size=(2, 2))(conv3)
    concat4 = Concatenate()([up4, conv2])
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(concat4)
    conv4 = Conv2D(128, 3, activation='relu', padding='same')(conv4)
    conv4 = se_block(conv4)
    
    up5 = UpSampling2D(size=(2, 2))(conv4)
    concat5 = Concatenate()([up5, conv1])
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(concat5)
    conv5 = Conv2D(64, 3, activation='relu', padding='same')(conv5)
    
    # Output: LAB color space (3 channels: L, A, B)
    outputs = Conv2D(3, 1, activation='tanh')(conv5)
    
    model = Model(inputs=[inputs_sar, inputs_optical], outputs=outputs)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

# Load the data (SAR and optical)
print("Loading the data (SAR and optical)")
base_path = 'v_2'
sar_files, optical_files = load_dataset_paths(base_path)

# Train/test split
print("Splitting the data into train and validation sets")
X_train_sar, X_val_sar, X_train_optical, X_val_optical = train_test_split(
    sar_files, optical_files, test_size=0.2, random_state=42
)

# Define and train the model
print("Defining and training the model")
input_shape = (256, 256, 3)  # Assuming RGB images of size 256x256
model = advanced_unet_with_fusion(input_shape)

# Create data generators
batch_size = 16
train_generator = CustomDataGenerator(X_train_sar, X_train_optical, batch_size)
val_generator = CustomDataGenerator(X_val_sar, X_val_optical, batch_size)

# Train the model
print("Training the model")
model.fit(train_generator, validation_data=val_generator, epochs=20)

# Save the trained model
print("Saving the trained model")
model.save('advanced_sar_colorization_model.h5')